# CoreMS Tutorial
`CoreMS` framework for mass spectrum

### 1. Create the reader obj 

In [5]:
import os

from corems.mass_spectrum.input.boosterHDF import ReadHDF_Booster
from corems.mass_spectrum.input.textMassList import ReadMassList
from corems.transient.input.BrukerSolarix import ReadBrukerSolarix

file_location = os.path.join(os.getcwd(), "tests/tests_data/") + "ESI_NEG_ESFA.ascii"
polariy = -1
reader_obj = ReadMassList(file_location, polariy, delimiter="  ", isCentroid=True)
#reader_obj = ReadMassList(file_location, polariy, delimiter=",")
#reader_obj = ReadMassList(file_location, polariy, delimiter="\t")
#reader_obj = ReadHDF_Booster(file_location, polariy) #acess MassSpec Obj  

### 2. Create the mass spectrum obj

In [6]:
mass_spectrum = reader_obj.get_mass_spectrum(auto_process=True)
mass_spectrum.mz_exp, mass_spectrum.abundance

(array([153.6634656, 153.9386664, 154.7982007, ..., 998.1886882,
        998.6415286, 999.1209952]),
 array([1844007, 1185563, 1175600, ..., 1566755, 1512993, 1760800],
       dtype=int64))

### 3. Access mass spectrum peak objs

In [12]:
for mspeak in mass_spectrum.sort_by_abundance():
    print(mspeak.mz_exp, mspeak.abundance, mspeak.resolving_power, 
          mspeak.signal_to_noise, mspeak.kdm, mspeak.is_assigned)
    

170.8382726 1169732 1171201 4.0
176.7010883 1169962 1153831 4.0
172.2127467 1169968 1650517 4.0
176.41590159999998 1170437 957619 4.0
177.2349283 1170494 982304 4.0
165.2276735 1170555 1383920 4.0
167.5103234 1170844 1133574 4.0
172.215181 1170936 818070 4.0
177.58872069999998 1171019 1273578 4.0
174.6766608 1171022 1235281 4.0
158.713096 1171104 1037668 4.0
165.05450009999998 1171281 943472 4.0
167.3759726 1171322 1216311 4.0
176.3754733 1171569 1194075 4.0
180.7350177 1171630 865015 4.0
172.3690789 1171752 914298 4.0
172.6788558 1171752 1385047 4.0
168.6688322 1171819 1316961 4.0
176.0517352 1171852 1069426 4.0
167.72495859999998 1172273 1147842 4.0
180.9088812 1172304 994602 4.0
178.6779431 1172525 967723 4.0
179.7708817 1172544 858039 4.0
178.7858689 1172591 1052116 4.0
172.7165803 1172665 1283828 4.0
178.1703699 1172765 1079112 4.0
179.93623630000002 1172786 1085581 4.0
171.8160625 1172787 1453892 4.0
172.54354840000002 1172788 946360 4.0
166.35576899999998 1173014 1361717 4.0
176

165.9447155 1227277 1400356 4.3
182.6985397 1227392 741574 4.3
183.4982778 1227474 1255561 4.3
167.72571269999997 1227539 900335 4.3
204.25569950000002 1227757 1164311 4.0
175.6901635 1227838 1009350 4.3
168.9274024 1227932 1418412 4.3
170.31954030000003 1227973 968339 4.3
205.4574604 1227999 1020696 4.0
181.0874884 1228045 1154982 4.3
180.00491730000002 1228065 903751 4.3
201.29629880000002 1228112 1146188 4.1
203.7083279 1228157 762700 4.0
176.2319583 1228244 872865 4.3
195.8443362 1228335 981229 4.1
201.508023 1228358 1006145 4.0
203.9738167 1228493 1215889 4.0
179.590879 1228764 1609118 4.3
166.71634519999998 1228857 958710 4.3
182.2801384 1229031 1212407 4.3
199.457974 1229031 1182126 4.1
180.5655259 1229118 1204254 4.3
201.20799250000002 1229188 793177 4.1
182.6172536 1229230 867199 4.3
196.5344337 1229282 1198688 4.1
198.71348899999998 1229417 853329 4.1
170.49802160000002 1229467 1372060 4.3
195.9323778 1229536 709943 4.1
190.7087072 1229551 1148643 4.2
192.1637909 1229618 1130

179.41590290000002 1277585 1065682 4.5
199.82020690000002 1277618 820783 4.3
183.9023823 1277634 1207812 4.5
222.68347409999998 1277640 914379 4.0
173.64140669999998 1277767 900304 4.6
183.9754122 1277788 1259218 4.5
184.1874759 1277835 1069277 4.5
215.6771636 1278002 1089126 4.1
195.0427592 1278199 931398 4.4
199.6240387 1278255 887009 4.3
181.0711552 1278425 1164097 4.5
187.4122362 1278478 1112614 4.5
199.77617619999998 1278509 940356 4.3
173.0785584 1278580 1330149 4.6
218.6057883 1278607 747148 4.1
197.8693332 1278616 802724 4.3
205.6344011 1278915 913226 4.2
193.6285353 1279097 870881 4.4
221.77721090000003 1279145 920130 4.0
217.68688430000003 1279615 1027769 4.1
178.9223544 1279663 1654907 4.6
212.5752781 1279782 565652 4.2
191.08541369999998 1279794 972666 4.4
175.7115464 1280152 862228 4.6
218.6890067 1280261 934048 4.1
175.6522427 1280379 981751 4.6
182.1206246 1280391 1050032 4.5
171.0509828 1280470 1129600 4.6
177.2424471 1280535 1271330 4.6
176.6846217 1280648 1753746 4.6


192.6828676 1331970 996830 4.7
227.265758 1332245 953716 4.2
199.9717675 1332272 999044 4.6
203.3677835 1332331 1029240 4.5
171.11194730000003 1332356 954298 4.8
198.6146003 1332569 1018026 4.6
207.62817080000002 1332928 1043410 4.5
195.3892937 1333233 1079519 4.6
180.4545051 1333236 1227075 4.8
202.48558880000002 1333243 1038682 4.5
192.8937199 1333420 1283400 4.7
206.1302934 1333651 1002513 4.5
174.0384531 1334131 1114276 4.8
178.2832659 1334334 981179 4.8
177.58730380000003 1334371 963874 4.8
226.54766150000003 1334410 1023953 4.2
179.7110669 1334564 938617 4.8
206.6790903 1334568 650460 4.5
214.2015946 1334610 1123611 4.4
166.1383311 1334634 1053668 4.8
159.601772 1334841 1058734 4.8
225.28622919999998 1334873 1196721 4.2
183.1762202 1334884 1230592 4.8
230.15654790000002 1334974 617404 4.2
190.2480064 1335008 750954 4.7
199.489054 1335104 772874 4.6
193.2015569 1335161 779516 4.7
221.24284169999999 1335173 1181928 4.3
191.4766936 1335282 992394 4.7
219.4205785 1335349 1035387 4.3


184.3075294 1400648 892439 5.1
253.21445669999997 1400663 917590 4.1
174.48254509999998 1401837 1121665 5.2
187.3443018 1402221 1191783 5.1
213.5444171 1403000 1010690 4.7
216.5047837 1404354 1187575 4.7
212.1000727 1404425 979941 4.8
224.0525804 1404602 881122 4.6
210.70059390000003 1404693 1153272 4.8
218.3622752 1404725 752698 4.7
255.7653435 1404876 672850 4.1
218.5635131 1404932 939114 4.7
224.60115580000002 1404966 1020860 4.6
259.7386894 1405169 750870 4.0
212.66065409999996 1405308 996079 4.8
244.4276099 1405483 775064 4.3
216.8826176 1405765 965032 4.7
179.5735841 1405768 1039746 5.2
169.9687112 1406079 1008926 5.2
252.4891589 1406250 871035 4.1
216.87678590000002 1406527 911865 4.7
215.05916780000004 1406587 951142 4.7
180.25271850000001 1406983 1041035 5.2
184.0819541 1406996 1313542 5.2
197.02211540000002 1406998 1060246 5.0
248.4018915 1407036 677414 4.2
165.2973783 1407161 1196287 5.2
185.9712895 1407185 1009127 5.1
208.408435 1407280 881715 4.8
218.80017650000002 1407607

...

465.4011546 14678022 375243 43.1
339.3395443 14688385 493646 45.2
431.5700096 14690366 419956 43.1
274.2819352 14693313 625087 57.5
305.3107643 14696115 1503754 51.3
576.4617561000001 14696483 318023 40.8
324.328048 14703057 499390 47.2
474.4334005 14703389 359750 43.2
561.4320938 14705055 303915 39.9
575.3940007 14708711 319721 40.9
393.4138802 14711796 448879 42.6
444.4123869 14716722 387899 43.4
361.4471305 14720880 504897 43.8
421.39277000000004 14729695 412464 43.0
287.35387819999994 14731614 657173 55.4
537.5063158 14740566 315521 40.9
332.3071 14742298 588901 46.4
344.3467113 14763740 546833 44.5
529.4859046 14768345 340684 41.8
349.4929563 14779539 901008 44.2
382.3620863 14786575 443532 43.1
215.2023795 14796173 893098 68.4
385.3346004 14801803 445100 42.9
342.36368369999997 14809416 530216 44.6
231.18717059999997 14813303 814181 66.3
298.30215530000004 14814015 608552 52.8
396.37900110000004 14814902 460094 42.6
323.4860111 14815008 535182 47.6
361.368355 14821192 514352 44.1

245.2583378 17654262 791708 75.6
438.4568451 17677342 410686 52.2
502.42636460000006 17681346 357582 51.4
490.4241237 17683030 370809 51.8
544.4755318 17694102 333997 49.4
445.5343371999999 17719232 394153 52.6
609.5723825 17732604 292182 53.5
635.504912 17739416 282248 55.0
641.4549226 17741402 271289 55.0
273.32182830000005 17751544 705922 69.8
361.3643757 17760070 525451 53.2
322.3916873 17763232 590976 58.6
557.5676925 17767094 318940 48.5
360.3318876 17767736 499880 53.2
633.453456 17797602 275149 55.2
544.4390646 17808768 321249 49.7
583.3973154 17813082 307518 51.7
282.2724978 17831674 663174 68.3
340.33766549999996 17836912 539626 55.2
243.23663430000005 17839182 790853 77.3
267.3616032 17842904 686307 71.1
555.6099305 17843930 327179 48.8
298.3304178 17845074 620967 64.0
393.38140319999997 17845228 451876 52.0
406.3775605 17852174 427695 51.9
298.2855248 17861428 598644 64.0
477.6078607 17865796 381916 52.6
448.47373799999997 17879334 394977 53.1
635.4686574 17890982 282787 55

497.420034 25487166 361531 75.4
308.3335017 25503648 616889 88.5
299.46119330000005 25513554 639333 92.2
603.4182125 25530668 299851 77.8
345.3659693 25532844 519620 78.0
354.3456839 25534548 523346 77.6
551.4486816000001 25538038 317236 70.5
362.35724389999996 25555252 500608 76.7
427.5732153 25556438 425596 75.7
593.4368055 25559870 308757 74.9
593.5093469999999 25576226 306666 74.9
605.433187 25578744 294673 78.0
279.2302513 25612562 688491 98.9
336.3113242 25616240 543365 80.0
591.4217297 25634342 304969 75.1
350.3474695 25695932 527704 78.1
351.3758528 25696584 525245 78.1
595.4308101 25706518 302119 75.3
411.36661349999997 25774806 433374 76.0
215.26000860000002 25789896 799707 120.7
394.33911989999996 25808084 456778 75.4
446.42081449999995 25837612 394179 77.5
495.4557772 25855416 361367 76.5
326.32572680000004 25903156 593331 84.5
231.2201989 25910422 842445 117.4
619.4803547 25931202 293997 79.1
565.426212 25978774 311255 73.4
525.5636682 26021876 349404 74.9
437.3802659 2602

461.4987983 44938908 383020 136.1
346.34099560000004 45063248 527811 138.9
301.3332954 45077356 639653 164.3
318.3200872 45106848 544535 151.5
334.3302668 45119840 551755 145.2
299.33071960000007 45141496 644395 164.5
547.5113659 45262052 330709 128.8
325.33679930000005 45301004 526187 149.0
487.48538909999996 45416620 376406 136.3
429.45936969999997 45459196 413159 135.9
577.4456852999999 45486140 322882 129.8
479.40569800000003 45513976 388623 136.6
225.1680852 45563236 860812 210.3
493.4971128 45679696 361174 136.4
385.50130310000003 45687740 471185 135.7
541.4282437999999 45700632 331773 130.1
221.18535409999998 45805964 639116 212.6
495.5132246 46155204 364832 137.9
555.4430428999999 46282372 329748 129.0
427.5003257 46319048 422738 138.6
255.27361340000002 46391928 719673 194.9
577.4820761 46403644 320672 132.5
227.22304559999998 46404972 847920 213.0
575.4302822000001 46419944 321504 132.5
551.4697723999999 46444036 332421 129.5
399.4322067 46477668 458059 137.1
359.5026871 4654

341.5132236 210808544 570681 669.6
393.402632 211028976 495265 632.3
327.39070780000003 212206480 596971 703.8
297.2613269 215689360 647428 792.5
349.3796385 216122448 552317 668.5
379.3773873 220363856 502787 664.9
377.3967106 221510064 518266 668.3
365.3512648 222881472 524846 681.2
281.2482595 223642880 699377 877.8
279.2667985 223772960 679319 878.3
353.3415439 224972288 541815 695.9
361.3531492 234676032 534088 721.7
365.3876107 235857616 533227 720.9
337.3691006 238428832 581964 757.5
311.29182280000003 239161504 620841 844.4
349.3431828 242795568 556636 751.2
339.3500756 244131984 578476 775.6
307.292737 245122176 638850 865.5
345.3449174 249477536 566229 776.3
363.3703877 252568352 533731 772.1
367.3684341 255912528 533750 782.4
389.36859189999996 261425136 498982 783.7
335.3515499 263167104 581366 854.6
325.32131180000005 264436992 603016 877.4
339.3136867 265332848 577467 843.1
383.3395208 267387200 511356 807.1
267.25236040000004 267479712 699573 1091.6
293.2983918 279224000

### 4. Access molecular formula  objs

In [14]:
for mspeak in mass_spectrum.sort_by_abundance():
   if mspeak.is_assigned: #or just if mspeak:
        for mf in mspeak:
            print(mf.mz_theor,mf.to_string,mf.to_string_formatted )